### Öneri Sistemleri

Kullanıcıdan hakkında edinilen bilgiler doğrultusunda, yada bir sisteme authentication olurken sizlerden alınan bilgiler dayanarak, öğeleri kullanıcılara tavsiye etmek içn kullanılan algoritmaların toplamına **öneri sistemleri** diyoruz. Bu sistemler tahmin edebileceğinizden daha fazla yerde bulunmaktadır. Online mağzalar(e-ticaret siteleri), film veritabanları (Netflix), spotify, youtube ve size yönelik gelen yani direk sizin özellikleirnize göre gelen iş ilanları (linkedin, kariyer vb.), tedarik zinciri gibi sistemlerde sıklıkla görülmektedir. 

*Öneri Sistemleri Temelde 2'ye ayrılır*<br>
**1.Content-Base:** İçerik tabanlı sistemlerde ki temel düşünce yapısı gayet basittir. Bir kullanıcı belirli bir içerikte film izlerse, kitap okursa, o kullanıcıya gene aynı içerikte bir film, kitap, video vb önerilir. Örneğin, burak yılmaz adlı kullanıcı Amazon'dan dünya savaş tarihi konusunda devamlı kitap satın almamktadır. Bu bağlamda Amazon'un bu kullanıcıya dünya savaş tarihi ile ilgil bir kitap önermesi çok muteberdir. Ayrıca bu kullanıcı özellikle bir yazarın kitaplarını Kindle vasıtasıyla yüksek bir şekilde derecelendirise, bu kullanıcıya ilk etapta ilgili yazarın dünya savaş tarihi kitapalrı önerilir. Hatta örneğimizi dahada derinleştirisek, bu kullanıcı aynı yazarın I.Dünya Savaşı ile ilgili kitabına 4 yıldız, Napolean Savaş tarihine ait kitabına 5 yıldız vermiş olsun, bu senaryoda bu kullancıya Napoleon Savaç tarihinden Waterloo kitabı önerilmesi çok yüksek bir ihtimaldir.<br>
**2.User-Base:** Kullanıcı tabanlı içerinde gene içerik tabanlı gibi çok basit bir çalışma prensibine dayanır. Şunu düşünmenizi istiyorum, 5 kişilik bir birleriyle çok iyi anlaşan bir arkadaş gurubu var, bu algoritmadaki düşünce bu arkadaş gurubundan 4 kişi Netflix'ten Outlaw filmini izlemişlerse, son kalan kişiye Netflix Outlaw filmini önermektedir. Çünkü düşünce yapısı şudur bu kişinin arkadaşları bu filmi izlemişler ve yüksek bir puan ile derecelendirmişlerse ilgili kişide yüksek bir oranda bu filmi beğenebilir. Bu mantıkta hareket ederek şunu düşünebilirsiniz, Youtube'ye authentication olmadığını düşünün, Youtube bize bazı içerikler önermektedir, bu içerikleri tam anlamıyle User-Base algoritmasına göre dizayn etmektedir. Türkiye'yi bir hub olarak düşünürsek Türkiye'de yaşayan insanların en çok like veridği videoları bize önermektedir.
<br>
**3.Hibrit Systems:** Bu yapıda *Content-Base* ve *User-Base* algoritmaları birleştirilir. Kullanıcı öneri sistemi yapısı kullanan bir sisteme authentication olmadığında ve olduğu durumdaki senaryolara göre hareket eder. Yani Hibrit tavsiye sistemleri, tamamlayıcı avantajlarından yararlanmak için iki veya daha fazla öneri stratejisini farklı şekillerde birleştirir. <br>

#### Amaç

Bu projede, elimizde film ve bu filmlere verilmiş rating içeren 2 adet veri seti bulunmaktadır. Dışarıdan gelen kullanıcının daha önceden derecelendirdiği puanladığı kişisel verisini kullanrak bizim veritabanımızda bulundurduğumuz filmleri ona önereceğiz. Burada kilit noktalardan bir tanesi film türlerine proje esnasında verdiğimiz ağırlıktır. Örneğin kullanıcı *Toy Story* filmine verdiği rate yüksek ise ve bu filmin türleri otomatik olarak üst sıralara çıkmaktadır. 

In [ ]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
cd data

C:\Users\burak\data


In [3]:
movies_df = pd.read_csv('movies.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
rating_df = pd.read_csv('ratings.csv')
rating_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


Aşağıdaki satırda yapılamk istenilen işlem gerçeği yansıtmasada çıkan output incelendiğinde, şu sonuç çıkmaktadır. *Title* sütunundan yılları aldık ve tablosunun sonuna year sütunu oluşturup oraya ekledik. Yapılan işlem temelde bu şekilde gözükmektedir, lakin gerçekten yıllar ordan sökülüp alınmadı. Satır satır ne yapıldığını inceleyelim.

1.İlk satırda parentezleri ile birlikte yıllar ile filim başlıklarını (title) bir birlerinden ayırdık ve elimize parantezlerde dahil olmamk üzere yıl bilgilsi aldık. Örneğin *(1995)* gibi. Ve Tablonun sonuna *year* sütunu açtık ve oraya ekledik.<br>
2.İkinci satırda ilk elde ettiğimiz parantezli yıllarda bulunan parantezlerden kurtulduk. Ve temiz bir şekilde *year* sütununa yılları ekledik.<br>
3.Üçüncü satırsa parentezler dahil olmak üzere *(1995)* buradaki her bir karakteri boşluk karakteri yerleştirdik. Burada *replace()* metodundan faydalandık. <br>
4.Dördüncü satırda *title* sütunuda film adları yanında oluşan 6 karakterlik boşluğu kaldırdık.<br>

**Not:** *lamda* expression çok önemli bir kavramdır. Buradaki kullanımında *title* sütunun bütün özelliklerini sütun başlık tipi dahil *x* atadık ve onu türettik. Böylelikle istediğimiz *x* yazarak title başlığına müdahele edebiliriz.

In [5]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))', expand=False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)', expand=False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip()) # title.strip() = x.strip()
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Aşağıdaki yapılan işlem, türleri bir liste biçiminde tutmak, içeriğe dayalı öneri sistmeleri tekniği açısından uygun olmadığı için, tür listesini her sütunun olası bir değerine karşılık gelen bir vektöre dönüştürmak için *One Hot Encoding* tekniğini kullanacağız. Bu kodlama kategorik verilerin beslenmesi yani matematiksel olarak desketlenmesi içn gereklidir. Bu durumda, her farklı türü 1 veya 0 olarak içeren sütunlarda saklayacağız. 1 değeri, bir filmin bir türe sahip olduğuna ve 0 değeri ilgili filmin o türe ait olmadığını gösterir.

*1.Adımda: Türleri ilk durumda ivedi bir şekilde kullamayacağımız için film veri çerçevesine yeni bir alana kopyaladık.*<br>
*2.Adımda: Kullanılan veri çerçevesindeki (movies_df) her bir **title** satırı için, bütün türler listesini tekrarlanacaktır. Böylelikle Her bir filme ait olan tür saptanacak ve saptandığı anda 1 değeri atanacak. Şayet ilgili filmin türler listesinde bulunan türlere karşılık gelen bir türü bulunamadı bu durumda da NaN değer atanacak. Yani veri çerçevemize bir göz gezdirirseniz **Jumanji** filminin **Animation** türüne sahip olmadığını göreceksiniz. Burada işlediğiz işleme göre bu durumda **Jumanji** filmi için ilgili kategori olamdığından **NaN** değer basılacak. Lakin veri setine tekrar bakarsanız **Advanture** türüne sahip olduğunu göreceksiniz, bu bağlamda **Advanture** için **1** değerini alacaktır.<br>
*3.Adımda: Yukarıdaki adımda filmlerin ait olmadığı türlerine **Nan** değer basmıştık. Matematiksel olarak yani bilgisayarın işlem mantığı gereği **NaN** matematiksel işleme sokulamayacak bir değer olduğundan, **NaN** değer yerine **0** değerini bastık.<br>

**Not:** Yukarıdaki adımları uygulamamk için *pandas* kütüphanesinden faydalandık. BU kütüphaneye ait özel methodlar kullandık. Örneğin *iterrows()*, *fillna()*.

**iterrows():** https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html <br>
**fillna():** https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html

In [7]:
moviesWithGenres_df= movies_df.copy()

for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
        
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,A,d,v,e,n,t,...,S,-,I,X,W,N,(,,g,)
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,Adventure|Children|Fantasy,1995,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,Comedy|Romance,1995,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,Comedy,1995,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Projenin önemli safhalarından biri olan dışarıdan gelen kullanıcıların izledikleri filmleri derecelendirdikleri data ile bizim elimizin altında bulunan *Rating* veri çerçevesindeki her satıra karşılık gelen en az bir film ile ilişkilendirilmiş bir kullanıcı kimliği oluşturmak istiyoruz. Bu bağlamda *rating* veri çerçevemizde bulunan ve kulalnıcıların bizim veri tabanımızda ki filmleri onaylama tarihlerini tutuğumuz *timestamp* sütununa gerek olmamaktadır. Bu doğruldutda *pandas* kütüphanesinin *drop()* metodundan yararlanarak bu sütunu siliyoruz. Projede ihtiyaç duyulmayacak olan bu sütunu silmemizdeki asıl amaç bu projenin daha performanslı çalışmasını sağlamak için yani *RAM Bellek* tasaruffu için bu sütunu veri setinden drop ettik.

**drop():** https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

In [8]:
ratings_df = rating_df.drop('timestamp', 1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


##### Content-Base Recommendation System

Bu teknik bir kullanıcının bir öğenin en sevdiği yönlerinin ne olduğunu bulmaya çalışır ve daha sonra bu yönleri kapsayan yahut bu yönler ile benzeşen öğeleri kullancıya önerir.

Aşağıda kullanıcın faklı bir sistemde derecelendirdiği verileri *json* formatında elde ettiğimizi düşünelim. Json formatında elimize gelen verileri veri biliminde ana veri yapısı olan dataframe çeviriyoruz.

**DataFrame():** https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

In [15]:
userInput = [
    {'title':'Breakfast Club, The', 'rating':5},
    {'title':'Toy Story', 'rating':3.5},
    {'title':'Jumanji', 'rating':2},
    {'title':'Pulp Fiction', 'rating':5},
    {'title':'Akira', 'rating':4.5}
]
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


Kullanıcı hakkında elde ettiğimiz data'ya (inputMovies) movieId ekleyelim. Bunu yapmamızdaki amaç bizim veri tabanımızda filmlerin ait oldukları Id ile kullanıcı hakkında elde ettiğimiz data içerisindeki filmleri eşleyerek benzer filmlere Id atamsı yapmaktır.

*1.Adımda: Kullanıcı hakkında elde ettiğimiz data (inputMovies) içerisinde bulunan filmler ile bizim veritabanımızdda bulunan filmleri isimlerinde eşliyoruz.<br>
*2.Adımda: Yukarıdaki eşleştirme sonucunda bizim veritabanında bulunan filmlerin ait oldğu Id bilgilerini **inputMovies** veri setinde denk gelen filmlere ekliyoruz.
3.Adımda: **inputMovies** veri çerçevesinde ihtiyaç duymadığımız sütunları drop ediyoruz.

**isin():** Veri çerçevelerini filtrelemek için pandas isin () yöntemi kullanılır. isin () yöntemi, belirli bir sütunda belirli veya birden Çok değere sahip satırların seçilmesine yardımcı olur ve ilgili dataframe'deki her öğenin değerlerde bulunup bulunmadığı kontorl eder.
*Doc:* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html

**merge():** DataFrame veya Series yapılarındaki nesnelerini veritabanlarında aşina olduğumuz "join" mantığı ile birleştirme işlemidir. Birleştirme sütunlarda veya dizinlerde yapılır. 
*Doc:* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html


**Not:** Panda, birleştirme türü işlemlerde Series veya DataFrame'i dizinler ve çeşitli ilişkisel cebir işlevleri ile çeşitli set mantığıyla kolayca birleştirmek için çeşitli olanaklar sağlar.
*Doc:* https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [16]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title']).tolist()]
inputMovies = pd.merge(inputId, inputMovies) # concat, join => ?
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


Aşağıdkai işlemde kullanıcı datası ile bizim ana datamızdaki bilgileri Id gibi spesifik bir bilgi kullanarak birleştirdik. Veri setinin yapısnı incelediğimizde fark edeceğiniz gibi 5 satırlık 39 sütunluk bir veri seti elde ettik. 

In [17]:
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,A,d,v,e,n,t,...,S,-,I,X,W,N,(,,g,)
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,Adventure|Children|Fantasy,1995,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,296,Pulp Fiction,Comedy|Crime|Drama|Thriller,1994,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1246,1274,Akira,Action|Adventure|Animation|Sci-Fi,1988,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,1968,"Breakfast Club, The",Comedy|Drama,1985,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Projedeki ihtiyaçlar doğrultusunda sadece kullanıcının elde ettiğimiz datasına ait türleri elde ettik. 
# Yani yukarıda oluşturduğumuz veri setinden türleri aldık. Bunun için diğer alanları drop ettik.
userMovies = userMovies.reset_index(drop=True)

userGenreTable = userMovies.drop('movieId',1).drop('title',1).drop('genres',1).drop('year',1)
userGenreTable

,A,d,v,e,n,t,u,r,|,i,...,S,-,I,X,W,N,(,,g,)
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Kullanıcının derecelendirme puanlarını bir listeye aldık.
inputMovies['rating']

0    3.5
1    2.0
2    5.0
3    4.5
4    5.0
Name: rating, dtype: float64

**Line 19 & Line 20**'de yapılan işlemlerin amacı fimlerin rating'lerini kullanırak türlerine göre ağırlık vermekti. Öneri sistemlerini anlatığım ilk satıra dönerseniz orada bir örnek vermiştirk. Burak kullanıcsı aynı yazarın Napolic War hakkında yazdığı bir kitaba 5 yıldız vermişti, I.Dünya savaşı hakkında yazdığı kitaba ise 4 yıldız vermişti. ALgoritma arka tarafta bunlara bir ağırlık vermketedir. Bu ağırlığa dayanarak burak kullanıcısına ilk önce aynı yazarın Napolic War hakkındaki bir kitabını daha önerir, ikinci olarak I.Dünya savaşı hakkında kitap önerir. Bu kapsamda bizde aşağıda 19'zuncu ve 20'ci satırlarda kullanıcı verisine göre elde ettiğimiz ayrı veri setlerindeki verileri kullanarak kullanıcının beğendi filmlerin ağırlıklarını oluşturuyoruz ki bu ağırlığa göre ona yeni filmler önerelim.

In [22]:
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
userProfile

A    10.0
d    20.0
v    10.0
e    20.0
n    10.0
t    10.0
u    10.0
r    20.0
|    20.0
i    15.0
m    18.0
a    20.0
o    18.0
C    15.5
h    10.5
l    10.5
y    15.5
F    10.0
s     5.5
R     0.0
c     4.5
D    10.0
T     5.0
H     0.0
M     0.0
S     4.5
-     4.5
I     0.0
X     0.0
W     0.0
N     0.0
(     0.0
      0.0
g     0.0
)     0.0
dtype: float64

In [25]:
# burada film id lerine göre türleri içeren bir veri seti oluşturduk
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
genreTable = genreTable.drop('movieId',1).drop('title',1).drop('genres',1).drop('year',1)
genreTable

,A,d,v,e,n,t,u,r,|,i,...,S,-,I,X,W,N,(,,g,)
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151697,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151701,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
151703,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
genreTable.shape

(34208, 35)

In [28]:
# Artık hazırlıklar tamamlandı ve ağırlıkları hesapkadık, lütfen bu işlemde kullanılan veri setlerini yukarıya 
# çıkarak tekrardan inceleyiniz. Her birini burada kullanmak için hazrıladık.
recomendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recomendationTable_df.head()

movieId
1    0.904040
2    0.782828
3    0.543771
4    0.644781
5    0.360269
dtype: float64

In [30]:
# Ağırlıkları çoktan aza sıraladık.
recomendationTable_df= recomendationTable_df.sort_values(ascending=False)
recomendationTable_df.head()

movieId
6902      1.000000
5018      1.000000
81132     0.984848
148775    0.983165
144350    0.983165
dtype: float64

In [31]:
# Son olarak kullanıcıya onun zevklerine göre izlemesi gerek 20 filmi öneiryoruz. 
movies_df.loc[movies_df['movieId'].isin(recomendationTable_df.head(20).keys())]

,movieId,title,genres,year
542,546,Super Mario Bros.,Action|Adventure|Children|Comedy|Fantasy|Sci-Fi,1993
664,673,Space Jam,Adventure|Animation|Children|Comedy|Fantasy|Sc...,1996
4923,5018,Motorama,Adventure|Comedy|Crime|Drama|Fantasy|Mystery|S...,1991
6252,6350,Laputa: Castle in the Sky (Tenkû no shiro Rapy...,Action|Adventure|Animation|Children|Fantasy|Sc...,1986
6793,6902,Interstate 60,Adventure|Comedy|Drama|Fantasy|Mystery|Sci-Fi|...,2002
8605,26093,"Wonderful World of the Brothers Grimm, The",Adventure|Animation|Children|Comedy|Drama|Fant...,1962
8942,26590,G.I. Joe: The Movie,Action|Adventure|Animation|Children|Fantasy|Sc...,1987
9218,27155,"Batman/Superman Movie, The",Action|Adventure|Animation|Children|Fantasy|Sc...,1998
9825,32031,Robots,Adventure|Animation|Children|Comedy|Fantasy|Sc...,2005
12021,54278,Underdog,Action|Adventure|Children|Comedy|Fantasy|Sci-Fi,2007


##### İçerik Tabanlı Sistemlerin Avantajları Dezavantajları

**Avantajları:**<br>
*1. Kullanıcın herhangi bir konudaki eğilimleri tam anlamıyla saptanır*<br>
*2. Kullanıcı için kişiselleştirilmiş içerikler sunulur(Spotify, Netflix, Youtube)*<br>
**Dezavantajları:**<br>
*1. Başkalarının bu ürün hakkında ne düşündüğünü dikkate almaz, bu nedenle hibrit olarak User-Base & Content-Base kullanılmalıdır.*br>
*2.Kullanıcının beğenmediği veya sevdiği öğenin özelliklerini belirlemek her zaman kolay değildir. Zahmetli bir iştir. Çok fazla veri elde etmek bir araya getirmek gerekmektedir.*